In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
import sys
sys.path.append("C:/Users/clementine.rosier/OneDrive - Ekimetrics/Documents/GitHub/dynamic_pricing")

%load_ext autoreload
%autoreload 2

In [3]:
from src.context_generator import ContextGenerator
from src.simulation_with_context import ContextualDemandSimulation
from src.context_bandit import ContextBandit

In [4]:
# Instanciate context
mu_context = [1,5]
sigma_context = [2,2]
n_discrete = [2,3]
context_generator = ContextGenerator(mu_context, sigma_context, n_discrete)

# Instanciate demand simulation
beta_c = [1,6] 
beta_d = [[1,-2], [6,8,-3]]
mu_e = 50
sigma_e = 1
demand_simulation = ContextualDemandSimulation(beta_c, beta_d, mu_e, sigma_e)

# Instanciate bandit
size_context= len(mu_context) + sum(n_discrete) + 1

k_p = [10,25,15,20,30,50,35,40,5]
norm_mean = 0
norm_std = 1
m_0 = np.zeros(shape=(len(k_p),size_context)) + norm_mean
q_0 = np.zeros(shape=(len(k_p),size_context)) + norm_std

bandit=ContextBandit(k_p, size_context, m_0, q_0)

ContextBandit model instanciated with 9 arms.


In [6]:
hist = []
# Exploration round
for i in range(300):
    for j in range(bandit.k):
        #bandit.chose_action(bandit_context, force_action=j)
        #price = k_p[bandit.action]
        # Simulate context and observe reward
        context_c, context_d = context_generator.simulate()
        bandit_context = np.array(context_c + [y for x in context_d for y in x] + [1])
        bandit.chose_action(bandit_context, force_action=j)
        price = k_p[bandit.action]
        reward = demand_simulation.evaluate(price, context_c, context_d) * price
        # Update params
        bandit.update(reward, bandit_context)

for i in range(100):
    # Simulate context and observe reward
    context_c, context_d = context_generator.simulate()
    bandit_context = np.array(context_c + [y for x in context_d for y in x] + [1])
    bandit.chose_action(bandit_context, method="thompson")
    price = k_p[bandit.action]
    reward = demand_simulation.evaluate(price, context_c, context_d) * price
    optimal_price=demand_simulation.get_optimal_price(context_c, context_d)
    optimal_revenue =-demand_simulation.compute_revenue(optimal_price,context_c, context_d)
    print(price,reward,optimal_price,optimal_revenue)
    
    # Update params
    bandit.update(reward, bandit_context)
    hist.append(bandit.action)
    

KeyboardInterrupt: 

In [ ]:
occ = sorted(Counter(hist).items(), key = lambda k : k[1])
for index, nb_occ in occ : 
    print(f"Bucket {index} -- Nb occ {nb_occ} -- Price {bandit.k_p[index]}")

In [ ]:
j=1
bandit.chose_action(bandit_context, force_action=j)
price = k_p[bandit.action]
# Simulate context and observe reward
context_c, context_d = context_generator.simulate()
bandit_context = np.array(context_c + [y for x in context_d for y in x] + [1])
reward = demand_simulation.evaluate(price, context_c, context_d) * price
# Update params
bandit.update(reward, bandit_context)

In [ ]:
demand_simulation.get_optimal_price(context_c, context_d)